# Загрузка и подготовка данных

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
import pymorphy2


import torch
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import  tqdm

In [2]:
data = pd.read_csv("test_data.csv")
pd.set_option('display.max_colwidth', None)

In [3]:
data.head(10)

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,manager,Ага
5,0,5,client,Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать
6,0,6,manager,Да мы работаем с компанией которая нам подливает поэтому спасибо огромное
7,0,7,manager,Как как бы уже до этого момента работаем все устраивает + у нас сопровождение поэтому
8,0,8,client,Угу а на что вы обращаете внимание при выборе
9,0,9,manager,Как бы нет


In [4]:
#функция для лемматизации
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
    words = text.split()
    res = list()
    for word in words:
        res.append(morph.parse(word)[0].normal_form)
        
    return " ".join(res)

In [5]:
data['lemm_text'] = data['text'].apply(lemmatize) #лемматизируем текст

data.drop('line_n', axis=1, inplace=True) #удалим ненужный столбец

In [6]:
#импортируем предобученную модель

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model.eval()
# model.cuda()  #if you have a GPU

#функция для получения эмбеддингов
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    #[print(f'key: {k}, val: {v}') for k,v in t.items()]
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings) 
    return embeddings[0].cpu().numpy()


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Комментарий**
    
Есть ощущение, что роли перепутаны. Или же мы рассматриваем случай, когда в компанию звонит клиент и наш менеджер отвечает. Мы все-таки будем извлекать реплики, маркированные как 'client'. В любом случае это легко исправить, поменяв маркировку роли (закомментированная строка).


In [7]:
#managers = data[data['role'] == 'manager'][['dlg_id','text','lemm_text']]

In [8]:
managers = data[data['role'] == 'client'][['dlg_id','text','lemm_text']]
managers

,dlg_id,text,lemm_text
1,0,Алло здравствуйте,алло здравствуйте
3,0,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться
5,0,Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать,угу ну возможно вы рассмотреть и другой вариант видеть это хороший практика сравнивать
8,0,Угу а на что вы обращаете внимание при выборе,угу а на что вы обращать внимание при выбор
11,0,Что для вас приоритет,что для вы приоритет
...,...,...,...
472,5,Так дмитрий,так дмитрий
473,5,Все записала тогда завтра ждите звонка,всё записать тогда завтра ждать звонок
475,5,По поводу виджетов и с ними уже обсудите конкретно продам,по повод виджет и с они уже обсудить конкретно продать
476,5,Все я вам высылаю счет и с вами на связи если будут вопросы можете писать на ватсапе,всё я вы высылать счёт и с вы на связь если быть вопрос мочь писать на ватсап


In [9]:
# функция для поиска индексов тех фраз, где встретилось "приветсвие" или "прощание"
#-idx - индекс текущего текста
#-idx_list -  список уже найденных индексов
#-phrase - текущий текст
#-base_embedding - базовый эмбеддинг, с которым ищем расстояние 
#-treshold - порог на косинусное расстояние

def find_idx(idx, idx_list, phrase, base_embedding, threshold):
    query_embedding = embed_bert_cls(phrase, model, tokenizer).reshape(1, -1)
    cos = cosine_similarity(query_embedding, base_embedding)
    if max(cos[0]) > threshold: #0.933
        idx_list.append(idx)
            
    return idx_list  

# Приветствие

In [10]:
#что хотим найти в тексте
greeting_words1 = ['приветcтвовать', 'здравствуйте']
greeting_words2 = ['добрый день', 'день добрый', 'добрый вечер', 'вечер добрый', 'доброе утро', 'утро доброе']

In [11]:
for i in range(len(greeting_words2)):
    greeting_words2[i] = lemmatize(greeting_words2[i])
greeting_words2    

['добрый день',
 'день добрый',
 'добрый вечер',
 'вечер добрый',
 'добрый утро',
 'утро добрый']

In [12]:
#эмбеддинги для приветствий, состоящих из одного слова (базовые)
greeting_embeddings1 = []
for greeting in greeting_words1:
    greeting_embeddings1.append(embed_bert_cls(greeting, model, tokenizer))
    
greeting_embeddings1 = np.array(greeting_embeddings1)
greeting_embeddings1.shape

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(2, 768)

In [13]:
#эмбеддинги для приветствий, состоящих из двух слов (базовые)
greeting_embeddings2 = []
for greeting in greeting_words2:
    greeting_embeddings2.append(embed_bert_cls(greeting, model, tokenizer))
    
greeting_embeddings2 = np.array(greeting_embeddings2)
greeting_embeddings2.shape

(6, 768)

In [14]:
#пройдемся по первым 15 предложениям каждого диалога, чтобы найти "приветствие"
idx_greeting = []
threshold = 0.933

for dlg_id in managers.dlg_id.unique():
    for i in tqdm(managers[managers['dlg_id']==dlg_id].index.tolist()[:15], desc=str(dlg_id)):
        text = managers['lemm_text'].loc[i].split()

        length = len(text)
        if length == 1:
            idx_greeting = find_idx(i, idx_greeting, text, greeting_embeddings1, threshold)
            if text[0] in ['добрый', 'доброе']: #отдельно обработаем короткое приветсвие
                idx_greeting.append(i)
        else: 
            for j in range(length): 
                idx_greeting = find_idx(i, idx_greeting, text[j], greeting_embeddings1, threshold)
                idx_greeting = find_idx(i, idx_greeting, text[j:j+2], greeting_embeddings2, threshold)

idx_greeting = set(idx_greeting)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


0:   0%|          | 0/15 [00:00<?, ?it/s]

1:   0%|          | 0/15 [00:00<?, ?it/s]

2:   0%|          | 0/15 [00:00<?, ?it/s]

3:   0%|          | 0/15 [00:00<?, ?it/s]

4:   0%|          | 0/14 [00:00<?, ?it/s]

5:   0%|          | 0/15 [00:00<?, ?it/s]

In [15]:
managers.loc[list(idx_greeting)]

,dlg_id,text,lemm_text
1,0,Алло здравствуйте,алло здравствуйте
166,2,Алло здравствуйте,алло здравствуйте
110,1,Алло здравствуйте,алло здравствуйте
250,3,Алло дмитрий добрый день,алло дмитрий добрый день
251,3,Добрый меня максим зовут компания китобизнес удобно говорить,добрый я максим звать компания китобизнес удобно говорить


In [16]:
data['greeting'] = False
data.loc[list(idx_greeting), 'greeting'] = True

# Прощание

**Комментарий**

Аналогично работаем с прощаниями

In [17]:
parting_words1 = ['свидание']
parting_words2 = ['до свидания', 'хорошего дня', 'хорошего вечера', 'хорошего дня', 'хорошего вечера']

In [18]:
for i in range(len(parting_words2)):
    parting_words2[i] = lemmatize(parting_words2[i])
parting_words2   

['до свидание',
 'хороший день',
 'хороший вечер',
 'хороший день',
 'хороший вечер']

In [19]:
parting_embeddings1 = []
for parting in parting_words1:
    parting_embeddings1.append(embed_bert_cls(parting, model, tokenizer))
    
parting_embeddings1 = np.array(parting_embeddings1)
parting_embeddings1.shape

(1, 768)

In [20]:
parting_embeddings2 = []
for parting in parting_words2:
    parting_embeddings2.append(embed_bert_cls(parting, model, tokenizer))
    
parting_embeddings2 = np.array(parting_embeddings2)
parting_embeddings2.shape

(5, 768)

In [21]:
#теперь пройдемся по последним 15 предложениям каждого диалога, чтобы найти "прощание"
idx_parting = []
threshold = 0.96
for dlg_id in managers.dlg_id.unique():
    for i in tqdm(managers[managers['dlg_id']== dlg_id].index.tolist()[-15:], desc=str(dlg_id)):
        text = managers['lemm_text'].loc[i].lower().split()
        length = len(text)
        if length == 1:
            idx_parting = find_idx(i, idx_parting, text, parting_embeddings1, threshold)
        else: 
            for j in range(length): 
            
                idx_parting = find_idx(i, idx_parting, text[j], parting_embeddings1, threshold)
                idx_parting = find_idx(i, idx_parting, text[j:j+2], parting_embeddings2, threshold)

idx_parting = set(idx_parting)

0:   0%|          | 0/15 [00:00<?, ?it/s]

1:   0%|          | 0/15 [00:00<?, ?it/s]

2:   0%|          | 0/15 [00:00<?, ?it/s]

3:   0%|          | 0/15 [00:00<?, ?it/s]

4:   0%|          | 0/14 [00:00<?, ?it/s]

5:   0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
managers.loc[list(idx_parting)]

,dlg_id,text,lemm_text
479,5,Ну до свидания хорошего вечера,ну до свидание хороший вечер
163,1,До свидания,до свидание
108,0,Всего хорошего до свидания,весь хороший до свидание
335,4,Во вторник все ну с вами да тогда до вторника до свидания,в вторник всё ну с вы да тогда до вторник до свидание


In [23]:
data['parting'] = False
data.loc[list(idx_parting),'parting'] = True

In [24]:
data

,dlg_id,role,text,lemm_text,greeting,parting
0,0,manager,Алло,алло,False,False
1,0,client,Алло здравствуйте,алло здравствуйте,True,False
2,0,manager,Добрый день,добрый день,False,False
3,0,client,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться,False,False
4,0,manager,Ага,ага,False,False
...,...,...,...,...,...,...
475,5,client,По поводу виджетов и с ними уже обсудите конкретно продам,по повод виджет и с они уже обсудить конкретно продать,False,False
476,5,client,Все я вам высылаю счет и с вами на связи если будут вопросы можете писать на ватсапе,всё я вы высылать счёт и с вы на связь если быть вопрос мочь писать на ватсап,False,False
477,5,manager,Спасибо спасибо,спасибо спасибо,False,False
478,5,manager,Да да тогда созвонимся ага спасибо вам давайте,да да тогда созвониться ага спасибо вы давать,False,False


In [25]:
#смотрим, в каких диалогах менеджер и поздоровался, и попрощался
set(managers.loc[list(idx_greeting)].dlg_id)&set(managers.loc[list(idx_parting)].dlg_id)

{0, 1}

# Имя

**Комментарий**

Здесь воспользуемся библиотекой natasha. Она умеет выделять имена собсвтенные в русском языке. Но не всегда делает это верно. Поэтому извлечение имени делаем в два этапа:

 * Сначала воспользумся методом NamesExtractor, который отсеет все, что не является именем собсвенным.
   <br><span style="color:red"> Проблема:</span> Может прихватить что-то лишнее. Например, короткие предлоги или союзы - "из", "по", "ли" и т.д</br> 
   
   
   
 * Слова, изъятые c помощью метода NamesExtractor, дополнительно проверим на часть речи. 
   <br><span style="color:red"> Проблема:</span> Захватывает слова (или звуки), которые можно встретить только в разговорной речи. К примеру, в тексте встретилось "че", и оно определялось как имя собственное.</br> Но самые короткие имена в русском языке - это "Ия" и "Ян". Учтем этот факт.
   
 
 Так мы найдем только имена собсвенные в тексте. Но это не значит, что менеджер представился. Поэтому мы будем запоминать индекс найденного имени в тексте и смотреть, что стоит перед ним или после него. 

In [26]:
#from natasha import NamesExtractor, MorphVocab


from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,

    MorphVocab,
    NamesExtractor,
                
    Doc
)

segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)


morph_vocab = MorphVocab()
extractor = NamesExtractor(morph_vocab)

In [27]:
#managers = data[data['role'] == 'client'][['dlg_id','text','lemm_text']]

In [28]:
key_words = ['это', 'звать', 'я'] #что ожидаем до или после имени собственного
idx_names = []
#d = {}
for idx, row in zip(managers.index, managers['lemm_text']):
    matches = extractor(row)
    for i, match in enumerate(matches):
        if match.fact.first != None:
            word = match.fact.first
            word = word.capitalize() #на этом этапе natasha ошибется, если имя не с заглавной буквы
            #print(word)
            doc = Doc(word)

            doc.segment(segmenter)
            doc.tag_morph(morph_tagger)
            doc.parse_syntax(syntax_parser)

            sent = doc.sents[0]
        
            
            try:
                previous_idx = row.split().index(match.fact.first)-1
                next_idx = row.split().index(match.fact.first)+1

                
                if (sent.morph.tokens[0].pos == 'PROPN' and (len(sent.morph.tokens[0].text)>2)           \
                                                            or sent.morph.tokens[0].text in ["Ия","Ян"])  \
                                                        and (row.split()[previous_idx] in key_words        \
                                                        or row.split()[next_idx] in key_words):
                    print(f'Имя менеджера - {sent.morph.tokens[0].text}', f'Индекс диалога: {managers.loc[idx].dlg_id}', '-'*30, sep='\n')
                    #d[managers.loc[idx].dlg_id] = sent.morph.tokens[0].text
                    idx_names.append(idx)
            except :
                continue


#idx_names = set(idx_names)

Имя менеджера - Ангелина
Индекс диалога: 0
------------------------------
Имя менеджера - Ангелина
Индекс диалога: 1
------------------------------
Имя менеджера - Ангелина
Индекс диалога: 2
------------------------------
Имя менеджера - Максим
Индекс диалога: 3
------------------------------
Имя менеджера - Анастасия
Индекс диалога: 5
------------------------------


**Комментарий**

Объединяя этот пункт и два предыдущих, можем сделать вывод, что только менеджер Ангелина поздоровалась и попрощалась в нулевом и первом диалогах.

In [29]:
managers.loc[list(idx_names)]

,dlg_id,text,lemm_text
3,0,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться
111,1,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление а мы сель обратить внимание что у вы срок заканчиваться
167,2,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мастер мы с вы сотрудничать по видео там
251,3,Добрый меня максим зовут компания китобизнес удобно говорить,добрый я максим звать компания китобизнес удобно говорить
338,5,Да это анастасия,да это анастасия


# Наименование организации

In [30]:
from yargy import Parser, rule, or_
from yargy.interpretation import fact
from yargy.predicates import gram, is_capitalized
from yargy.pipelines import morph_pipeline

In [31]:
Name = fact( 
    'Name', 
    ['surname', 'name', 'orgnm', 'abbr'])

In [32]:
ORGFORM = morph_pipeline([
                       'ооо',
                       'ип',
                       'компания',
                       'организация'
])
NAME = gram('Name').interpretation(Name.name)
SURN = gram('Surn').interpretation( 
    Name.surname)
ABBR = (is_capitalized()).interpretation( 
    Name.abbr)
ORGNAME = gram('NOUN').interpretation(Name.orgnm) 

In [33]:
ORGANIZATION = or_( 
    rule(ORGFORM, NAME, SURN),
    rule(ORGFORM, SURN, NAME),
    rule(ORGFORM, ORGNAME, ORGNAME),
    rule(ORGFORM, ORGNAME),
    rule(ORGFORM, ABBR),
    rule(ORGFORM, NAME, SURN, ABBR),
    rule(ORGFORM, ORGNAME, ABBR)
).interpretation(Name) 

In [34]:
orgparser = Parser(ORGANIZATION)

def orgs_extract(text, parser):
    for match in parser.findall(text):
        found_values = match.tokens
        company = [i.value for i in found_values]
    try:
        return company
    except:
        return 


In [35]:
rows_with_companies = {}
for idx, row in zip(managers.index, managers['lemm_text']):
    ans = orgs_extract(row, orgparser)
    if ans != None:
        rows_with_companies[idx] = orgs_extract(row, orgparser)
    

In [36]:
rows_with_companies

{3: ['компания', 'диджитал', 'бизнес'],
 111: ['компания', 'диджитал', 'бизнес'],
 167: ['компания', 'диджитал', 'бизнес'],
 251: ['компания', 'китобизнес']}

In [37]:
managers.loc[list(rows_with_companies.keys())]

,dlg_id,text,lemm_text
3,0,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться
111,1,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление а мы сель обратить внимание что у вы срок заканчиваться
167,2,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мастер мы с вы сотрудничать по видео там
251,3,Добрый меня максим зовут компания китобизнес удобно говорить,добрый я максим звать компания китобизнес удобно говорить
